In [1]:
from IPython.display import display, HTML

display(
    HTML("""
<style>
* {font-family:D2Coding;}
div.container{width:87% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input { font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:3px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="5" color="red">ch14_웹데이터 수집</font></b>

# 1절. BeautifulSoup과 parser

```pip install bs4``` 아나콘다를 설치하면 7500개여개의 패키지 설치
- [BeautifulSoup 공식사이트] : https://www.crummy.com/software/BeautifulSoup/
- [BeautifulSoup 공식문서] : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [23]:
# https://pypi.org/project/requests/
import requests # HTTP 요청 처리 lib

# https://pypi.org/project/requests-file/
from requests_file import FileAdapter

In [43]:
s = requests.Session() # HTTP 요청관리를 위한 세션 객체
s.mount("file://", FileAdapter())
# print(s)
response = s.get('file:///Volumes/DATA/mbc/ai_x/source/01_python/data/ch14_sample.html')
print(response)

<Response [200]>


In [44]:
response.status_code
#200 : 정상
#404 : 없는페이지 오류
#406 : get, post 오류
#500 : 파이썬 문법 오류

200

In [45]:
response.content # 바이너리 형식의 내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n  <head>\r\n    <meta charset="UTF-8" />\r\n  </head>\r\n  <body>\r\n    <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>\r\n    <h1 class="css">Hi, CSS</h1>\r\n    <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n    <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n    <div class="contents">\r\n      \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n      <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span\r\n      >\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n    </div>\r\n    <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb

In [46]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n  <head>\r\n    <meta charset="UTF-8" />\r\n  </head>\r\n  <body>\r\n    <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>\r\n    <h1 class="css">Hi, CSS</h1>\r\n    <div id="subject">subject 선택자 안의 내용</div>\r\n    <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n    <div class="contents">\r\n      선택자를 어떻게 작성하느냐에 따라\r\n      <span>다른<b>요소가 반환</b></span\r\n      >됩니다\r\n    </div>\r\n    <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n  </body>\r\n</html>\r\n'

In [47]:
# decode('utf-8') 와 같아서,
# 앞으로 이걸로 크롤링을 할 예정
response.text

'<!DOCTYPE html>\r\n<html lang="en">\r\n  <head>\r\n    <meta charset="UTF-8" />\r\n  </head>\r\n  <body>\r\n    <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>\r\n    <h1 class="css">Hi, CSS</h1>\r\n    <div id="subject">subject 선택자 안의 내용</div>\r\n    <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n    <div class="contents">\r\n      선택자를 어떻게 작성하느냐에 따라\r\n      <span>다른<b>요소가 반환</b></span\r\n      >됩니다\r\n    </div>\r\n    <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n  </body>\r\n</html>\r\n'

In [51]:
# html 파싱
from bs4 import BeautifulSoup
# 객체 생성
soup = BeautifulSoup(response.content, "html.parser") # 또는 response.text

soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
</head>
<body>
<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
<h1 class="css">Hi, CSS</h1>
<div id="subject">subject 선택자 안의 내용</div>
<p>CSS 선택자는 다양한 곳에서 활용됩니다</p>
<div class="contents">
      선택자를 어떻게 작성하느냐에 따라
      <span>다른<b>요소가 반환</b></span>됩니다
    </div>
<div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>
</body>
</html>

In [56]:
# soup.select_one('선택자') : 해당 선택자 처음 하나만
e1 = soup.select_one('h1'); # ★ 처음 나오는 h1태그 하나만
print('e1 :', e1) # ★
print('e1.text :', e1.text) # ★
print('e1.string :', e1.string)
print('e1의 속성들 :', e1.attrs)
print('e1의 title 속성 :', e1.attrs['title'])
print('e1의 title 속성 :', e1.attrs.get('title')) # ★ 딕셔너리는 이렇게 가져 오는 것이 더 좋다.
print('e1의 이름 :', e1.name)

e1 : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
e1.text : Hello, CSS
e1.string : Hello, CSS
e1의 속성들 : {'class': ['greeting', 'css'], 'id': 'text', 'title': 'greeting'}
e1의 title 속성 : greeting
e1의 title 속성 : greeting
e1의 이름 : h1


In [74]:
# soup.select('선택자') : 해당 선택자 모든 엘리먼트를 리스트로
el = soup.select('h1')


print('리스트 el : ', el)
print('el의 text들 : ',[e.text for e in el])
# for e in el:
#     print(e.text, end=',')
print('el의 속성들 : ',[e.attrs for e in el])
print('el의 class속성들 : ',[ e.attrs.get('class') for e in el ])

리스트 el :  [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
el의 text들 :  ['Hello, CSS', 'Hi, CSS']
el의 속성들 :  [{'class': ['greeting', 'css'], 'id': 'text', 'title': 'greeting'}, {'class': ['css']}]
el의 class속성들 :  [['greeting', 'css'], ['css']]


In [86]:
# soup.select_one(선택자) 와 soup.find(태그, 속성)
print('soup.select_one :', soup.select_one('h1.css'))
print('soup.find :', soup.find('h1',{'class':'css'}))
print('soup.find :', soup.find('h1', class_='css')) # class만 가능
print()
print('soup.select_one',soup.select_one('h1#text'))
print('soup.find',soup.find('h1', {'id':'text'}))
print()
print('soup.select')

soup.select_one : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
soup.find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
soup.find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>

soup.select_one <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
soup.find <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>

soup.select


In [101]:
# select(선택자)와 find_all(태그, 속성)
print('모든 h1.css, span 태그 (select) :', soup.select('h1.css, span'))
print('모든 h1.css, span 태그 (find_all) :', soup.find_all(['h1','span'], class_='css'))

모든 h1.css, span 태그 (select) : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
모든 h1.css, span 태그 (find_all) : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [100]:
# 없는 엘리먼트 찾기
print('find_all :', soup.find_all('a', class_='css')) # 빈 리스트
print('find :', soup.find('a', class_='css')) # None
print('select', soup.select('a.css'))
print('select_one', soup.select_one('a.css'))

find_all : []
find : None
select []
select_one None


<h1>2절. 정적 웹 데이터 수집 (정적 웹크롤링)</h1>

- json
- xml, html : select_one, select, find, find_all 가능

<h2>2-1. JSON파일</h2>

- requests모듈(get)
- urllib.request모듈(urlopen)

In [ ]:
# 크롤링 허용 범위는 사이트마다 ~/robots.txt 에서 확인할 수 있습니다.

In [240]:
# 방법1
import requests
import json
# response = requests.get('https://api.github.com')
# response, response.status_code
# response.text

In [239]:
# 방법2
from urllib.request import urlopen
# response = urlopen('https://api.github.com')
# response.status


In [236]:
text = '{"속성1":"값1", "속성2":"값2"}'

type(text)

str

In [237]:
# 문자(딕셔너리 타입)를 딕셔너리로
# '{"속성1":"값1", "속성2":"값2"}' => {"속성1":"값1", "속성2":"값2"}
import json
json.loads(text)

{'속성1': '값1', '속성2': '값2'}

## 2-2. html 파일

### 1) 환율정보 가져오기 (네이버 → 증권 → 시장지표)

In [134]:
import requests
from bs4 import BeautifulSoup
url = "https://finance.naver.com/marketindex/"
response = requests.get(url)
# response, response.status_code
# response.content (바이너리)
# response.content.decode('cp949') == response.text

soup = BeautifulSoup(response.text,'html.parser') # 또는 response.content
# soup

In [139]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
url = "https://finance.naver.com/marketindex/"
response = urlopen(url)
# response.status 상태코드
# response.read()
# response.read().decode('cp949')
soup = BeautifulSoup(response, 'html.parser') # 또는 response.read()
# soup

In [146]:
title = soup.select("h3.h_lst > span.blind")
[t.text for t in title]


['미국 USD', '일본 JPY(100엔)', '유럽연합 EUR', '중국 CNY', '달러/일본 엔', '유로/달러', '영국 파운드/달러', '달러인덱스', 'WTI', '휘발유', '국제 금', '국내 금']
['1,373.90', '952.28', '1,555.05', '190.92', '1.1342', '1.3520', '60.89', '1633.17', '3300.4']


In [166]:
price = soup.select("div.head_info > span.value")
[p.text for p in price]
[round(float(p.text.replace(',',''))) for p in price]
[round(float(''.join(p.text.split(',')))) for p in price]

[1374, 952, 1555, 191, 144, 1, 1, 99, 61, 1633, 3300, 146245]

In [158]:
# ',' 빼는 법

out = '1,373.90'
# 1번쨰
round(float(''.join(out.split(','))))

1374

In [160]:
# 2번쨰
round(float(out.replace(',','')))

1374

In [196]:
unit = soup.select('div.head_info > span > span.blind')
unit = [u.text for u in unit]
unit.insert(7, '') # 7번째 index에 '' 추가
unit

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [197]:
status = soup.select('div.head_info > span.blind')

[t.text for t in title]
[p.text for p in price]
unit
[s.text for s in status]

['하락', '하락', '하락', '하락', '상승', '하락', '하락', '상승', '하락', '하락', '하락', '상승']

In [188]:
len(title), len(price), len(unit), len(status)

(12, 12, 12, 12)

In [201]:
for idx in range(len(title)):
    print("{}.{} : {}{} - {}".format(idx+1, title[idx].text, price[idx].text, unit[idx], status[idx].text ))

1.미국 USD : 1,373.90원 - 하락
2.일본 JPY(100엔) : 952.28원 - 하락
3.유럽연합 EUR : 1,555.05원 - 하락
4.중국 CNY : 190.92원 - 하락
5.달러/일본 엔 : 144.3800엔 - 상승
6.유로/달러 : 1.1342달러 - 하락
7.영국 파운드/달러 : 1.3520달러 - 하락
8.달러인덱스 : 99.4200 - 상승
9.WTI : 60.89달러 - 하락
10.휘발유 : 1633.17원 - 하락
11.국제 금 : 3300.4달러 - 하락
12.국내 금 : 146244.92원 - 상승


In [206]:
# zip 함수 이용
for t, p, u, s in zip(title, price, unit, status) :
    print("{}: {}{} - {}".format(t.text, p.text, u, s.text))

미국 USD: 1,373.90원 - 하락
일본 JPY(100엔): 952.28원 - 하락
유럽연합 EUR: 1,555.05원 - 하락
중국 CNY: 190.92원 - 하락
달러/일본 엔: 144.3800엔 - 상승
유로/달러: 1.1342달러 - 하락
영국 파운드/달러: 1.3520달러 - 하락
달러인덱스: 99.4200 - 상승
WTI: 60.89달러 - 하락
휘발유: 1633.17원 - 하락
국제 금: 3300.4달러 - 하락
국내 금: 146244.92원 - 상승


In [208]:
for idx, (t, p, u, s) in enumerate(zip(title, price, unit, status)) :
    print("{}. {}: {}{} - {}".format(idx+1,t.text, p.text, u, s.text))

1. 미국 USD: 1,373.90원 - 하락
2. 일본 JPY(100엔): 952.28원 - 하락
3. 유럽연합 EUR: 1,555.05원 - 하락
4. 중국 CNY: 190.92원 - 하락
5. 달러/일본 엔: 144.3800엔 - 상승
6. 유로/달러: 1.1342달러 - 하락
7. 영국 파운드/달러: 1.3520달러 - 하락
8. 달러인덱스: 99.4200 - 상승
9. WTI: 60.89달러 - 하락
10. 휘발유: 1633.17원 - 하락
11. 국제 금: 3300.4달러 - 하락
12. 국내 금: 146244.92원 - 상승


### 2) 이번주 로또번호 출력

- https://dhlottery.co.kr/gameResult.do?method=byWin&wiselog=H_C_1_1 

```
1173회(2025년 05월 24일 추첨)
당첨번호 [1 5 18 20 30 35]
보너스 3
```

In [285]:
import requests
from bs4 import BeautifulSoup

url = 'https://dhlottery.co.kr/gameResult.do?method=byWin&wiselog=H_C_1_1'
response = requests.get(url)
# response.status_code

soup = BeautifulSoup(response.text,'html.parser')

title = soup.select_one('div.win_result > h4 > strong').text
desc = soup.select_one('div.win_result > p.desc').text
title1 = soup.select_one('div.win_result > div.nums > div.win strong').text
ball = soup.select('div.win_result > div.nums > div.win p > span')
ball = [b.text for b in ball]

title2 = soup.select_one('div.win_result > div.nums > div.bonus > strong').text
bonus = soup.select_one('div.win_result > div.nums > div.bonus > p > span').text

print('{} {}'.format(title, desc))
print('{} {}'.format(title1, ball))
print('{} {}'.format(title2, bonus))

1173회 (2025년 05월 24일 추첨)
당첨번호 ['1', '5', '18', '20', '30', '35']
보너스 3


In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://dhlottery.co.kr/gameResult.do?method=byWin&wiselog=H_C_1_1'
response = requests.get(url)
# response.status_code

soup = BeautifulSoup(response.text,'html.parser')

win_result = soup.find('div', class_='win_result')
# print(win_result)

times = win_result.find('strong').text

# times
date = soup.find('p',class_='desc').text

print(times, date)

num_win = soup.find('div',class_=['num','win'])
# num_win

title1 = num_win.find('strong').text
lotto_number = num_win.find_all('span') #배열

print(title1, [int(lotto.text) for lotto in lotto_number])

num_bonus = soup.find('div', class_=['bonus'])

# num_bonus
title2 = num_bonus.find('strong').text
bonus_number = num_bonus.find('span').text
print(title2, bonus_number)


1173회 (2025년 05월 24일 추첨)
당첨번호 [1, 5, 18, 20, 30, 35]
보너스 3


In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = 'https://dhlottery.co.kr/gameResult.do?method=byWin&wiselog=H_C_1_1'
response = urlopen(url)
# response.status

soup = BeautifulSoup(response,'html.parser')
# soup

title = soup.select_one('div.win_result > h4 > strong').text
desc = soup.select_one('div.win_result > p.desc').text
title1 = soup.select_one('div.win_result > div.nums > div.win strong').text
ball = soup.select('div.win_result > div.nums > div.win p > span')
ball = [b.text for b in ball]
title2 = soup.select_one('div.win_result > div.nums > div.bonus > strong').text
bonus = soup.select_one('div.win_result > div.nums > div.bonus > p > span').text

print('{} {}'.format(title, desc))
print('{} {}'.format(title1, ball))
print('{} {}'.format(title2, bonus))

1173회 (2025년 05월 24일 추첨)
당첨번호 ['1', '5', '18', '20', '30', '35']
보너스 3


In [25]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = 'https://dhlottery.co.kr/gameResult.do?method=byWin&wiselog=H_C_1_1'
response = urlopen(url)
soup = BeautifulSoup(response, 'html.parser')

win_result = soup.find('div', class_='win_result')
# print(win_result)

times = win_result.find('strong').text

# times
date = soup.find('p',class_='desc').text

print(times, date)

num_win = soup.find('div',class_=['num','win'])
# num_win

title1 = num_win.find('strong').text
lotto_number = num_win.find_all('span') #배열

print(title1, [int(lotto.text) for lotto in lotto_number])

num_bonus = soup.find('div', class_=['bonus'])

# num_bonus
title2 = num_bonus.find('strong').text
bonus_number = num_bonus.find('span').text
print(title2, bonus_number)

1173회 (2025년 05월 24일 추첨)
당첨번호 [1, 5, 18, 20, 30, 35]
보너스 3


### 3) 다음 검색 리스트

```
no   title                                                                  link
0    [비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채 안 늘리면 가능  https://v.daum.net/v/20250528103907230
1    [비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞                      https://v.daum.net/v/20250528075215864 
```

In [92]:
# 방법1
import requests
import pandas as pd
from bs4 import BeautifulSoup

word = '비트코인'
url = "https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q="+word

response = requests.get(url)
response, response.status_code

soup = BeautifulSoup(response.content,'html.parser')

items_find_list = [] # 검색한 결과를 담는 dict list

item_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')

len(item_el)

for idx, item in enumerate(item_el):
    # print(idx, item.text, item.attrs['href'])
    # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx, 'title':item.text, 'link':item.attrs.get('href')})

pd.DataFrame(items_find_list)


,no,title,link
0,0,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
1,1,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
2,2,“맥O날드보다 맛없어!” 혹평이 가득한 트럼프 만찬과 비트코인 피자데이[엠블록레터],http://v.daum.net/v/20250528143002735
3,3,"블랙록, 자사 비트코인 ETF 보유량 25% 확대…기관 투자 본격화 신호탄",http://v.daum.net/v/20250528151802080
4,4,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
5,5,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
6,6,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853
7,7,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
8,8,[비트코인 2025] 로빈후드 창업자 “토큰화 증권은 美 ‘자본 패권’ 키우는 수단”,http://v.daum.net/v/20250528110600761
9,9,"트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고",http://v.daum.net/v/20250528084245020


In [95]:

import requests
import pandas as pd
from bs4 import BeautifulSoup

word = '비트코인'

url = "https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&cluster_page=1&q="+word

response = requests.get(url)
response, response.status_code

soup = BeautifulSoup(response.text,'html.parser')

items_find_list = [] # 검색한 결과를 담는 2차원 리스트

item_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')

for idx, item in enumerate(item_el):
    items_find_list.append([idx, item.text, item.attrs.get('href')])

pd.DataFrame(items_find_list,columns=['no','title','link'])

,no,title,link
0,0,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
1,1,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
2,2,“맥O날드보다 맛없어!” 혹평이 가득한 트럼프 만찬과 비트코인 피자데이[엠블록레터],http://v.daum.net/v/20250528143002735
3,3,"블랙록, 자사 비트코인 ETF 보유량 25% 확대…기관 투자 본격화 신호탄",http://v.daum.net/v/20250528151802080
4,4,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
5,5,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
6,6,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853
7,7,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
8,8,[비트코인 2025] 로빈후드 창업자 “토큰화 증권은 美 ‘자본 패권’ 키우는 수단”,http://v.daum.net/v/20250528110600761
9,9,"트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고",http://v.daum.net/v/20250528084245020


In [ ]:
# 다음 뉴스 검색(키워드, 원하는 페이ㅣ)
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

keyword = '비트코인'
page = 2

# url = f"https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&q={keyword}&p={page}"
# print(url)
# response = requests.get(url)

url = f"https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y"
params = {'q':keyword,'p':page}
response = requests.get(url, params=params)
soup = BeautifulSoup(response.text,'html.parser')

items_find_list = [] # 검색한 결과를 담는 2차원 리스트
item_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')

for idx, item in enumerate(item_el):
    items_find_list.append({'no': (page-1)*10+idx, 'title':item.text, 'link':item.attrs.get('href')})
    print({'no': (page-1)*10+idx, 'title':item.text, 'link':item.attrs.get('href')})
    
# pd.DataFrame(items_find_list)

{'no': 10, 'title': ' 트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고 ', 'link': 'http://v.daum.net/v/20250528084245020'}
{'no': 11, 'title': ' 연일 최고가 경신하는 비트코인…투자자들 강 건너 불구경하는 이유는 ', 'link': 'http://v.daum.net/v/20250526212400275'}
{'no': 12, 'title': ' "비트코인 산다"…트럼프家, 25억달러 자금 조달 추진[코인브리핑] ', 'link': 'http://v.daum.net/v/20250528111837527'}
{'no': 13, 'title': ' [김대호 박사의 오늘의 키워드] 소비자신뢰지수·비트코인 2025·머스크 한마디·애플 괘씸죄 ', 'link': 'http://v.daum.net/v/20250528071325361'}
{'no': 14, 'title': ' 잘 나가는 비트코인 산다는데 왜?…트럼프 미디어 주가 10% 급락 [투자360] ', 'link': 'http://v.daum.net/v/20250528074002668'}
{'no': 15, 'title': ' "테슬라 팔고 갈아탔어요"…자녀계좌 보던 엄마들 \'돌변\' [마켓PRO] ', 'link': 'http://v.daum.net/v/20250528094602647'}
{'no': 16, 'title': ' 스트래티지, 비트코인 4020개 추가 매입...보유량 58만개 돌파 ', 'link': 'http://v.daum.net/v/20250527081116129'}
{'no': 17, 'title': ' 달러 스테이블코인, 국내 거래 확산…당국 외환흐름 통제력 약화 우려 ', 'link': 'http://v.daum.net/v/20250528152520516'}
{'no': 18, 'title': ' 비트코인과 엇갈리는 이더리움·리플, 왜? ', 'link': 'http://v.daum.net/v/20

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def collect_list(keyword, page=1):
    'keyword로 다음 검색한 결과(해당 page)를 return'
    url = f"https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y"
    params = {'q':keyword,'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text,'html.parser')

    items_find_list = [] # 검색한 결과를 담는 2차원 리스트
    item_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')

    for idx, item in enumerate(item_el):
        items_find_list.append({'no': (page-1)*10+idx, 'title':item.text, 'link':item.attrs.get('href')})

    return items_find_list

collect_list('비트코인',3)    
    
# pd.DataFrame(items_find_list)

[{'no': 20,
  'title': ' <단독>"예배 참석하면 코인 드려요. 5년 안에 비트코인 넘어섭니다"..신앙심 노린 코인 유사 사기 기승 ',
  'link': 'http://v.daum.net/v/20250528083323774'},
 {'no': 21,
  'title': ' 트럼프미디어, 비트코인 3.4조 투자 발표에 주가 10% 폭락 ',
  'link': 'http://v.daum.net/v/20250528080206055'},
 {'no': 22,
  'title': ' “가상화폐=애완용 돌” 직격했던 ‘월가황제’ 돌변…美 1위 은행도 주목한 ‘이것’ [월가그루픽] ',
  'link': 'http://v.daum.net/v/20250528153835219'},
 {'no': 23,
  'title': ' 비트레이어, 비트코인 채굴업체와 협력...“BTC 디파이로 나아갈 것” ',
  'link': 'http://v.daum.net/v/20250527114126672'},
 {'no': 24,
  'title': ' 뉴욕증시 휴장에 10.9만弗선 숨고른 비트코인 [매일코인] ',
  'link': 'http://v.daum.net/v/20250527103810840'},
 {'no': 25,
  'title': ' 스트래티지, 비트코인 4020개 추가 매입...보유량 58만개 돌파 ',
  'link': 'http://v.daum.net/v/20250527081116129'},
 {'no': 26,
  'title': ' "죽고 싶지 않으면 비트코인 비번 내놔"···외국인 납치·고문한 \'가상자산 왕\' ',
  'link': 'http://v.daum.net/v/20250528060036428'},
 {'no': 27,
  'title': ' [영상] “강세장 진입한 비트코인, 내년 2분기까지 최소 20만~30만 달러 갈 듯” ',
  'link': 'http://v.daum.net/v/20250527090137566'},
 {'no'

In [30]:
# 원하는 keyword 다음검색

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

result = []
keyword = "청바지"
pages = 3

for page in range(1, pages+1):
    result.extend(collect_list(keyword, page))
    time.sleep(3)
    
result_pd = pd.DataFrame(result)
result_pd
result_pd.head()

,no,title,link
0,0,[단독]대선 투표도 거르고 ‘외유성 출장’···대구지역 노사 대표들 해외 방문 논란,http://v.daum.net/v/20250527060110255
1,1,[팩트체크] 세로쓰기에서 가로쓰기까지…대선 투표용지 변천사,http://v.daum.net/v/20250528065513098
2,2,"“대선 투표 꼭…” 한국교회, 캠페인·기도회로 독려",http://v.daum.net/v/20250528030315081
3,3,"우범기 전주시장, 대선 투표 독려 캠페인 동참",http://v.daum.net/v/20250528110014351
4,4,"""근로자 대선 투표 보장""…광주상의·경총·TP, 실천 캠페인(종합)",http://v.daum.net/v/20250527173934357


In [34]:
keywords = ['청바지','면바지']
pages = 5
result0 = [] # 청바지 검색결과 50개
result1 = [] # 면바지 검색결과 50개

for i, keyword in enumerate(keywords):
    print(f'~~~{i}번째 {keyword} {page} 검색 중입니다.~~~')
    for page in range(1, pages+1):
        print(i, keyword, page)
        if i == 0:
            result0.extend(collect_list(keyword, page))
        else :
            result1.extend(collect_list(keyword, page))
        time.sleep(3)

~~~0번째 청바지 5 검색 중입니다.~~~
0 청바지 1
0 청바지 2
0 청바지 3
0 청바지 4
0 청바지 5
~~~1번째 면바지 5 검색 중입니다.~~~
1 면바지 1
1 면바지 2
1 면바지 3
1 면바지 4
1 면바지 5


In [35]:
result0_df = pd.DataFrame(result0)
result0_df.head()

,no,title,link
0,30,아이브 가을 ‘청바지핏 뽐내는 시구’ [MK포토],http://v.daum.net/v/20250527184503588
1,31,"'윤남기♥' 이다은, 다이어트 21kg 감량 효과 있네···완벽한 청바지 자태",http://v.daum.net/v/20250427195408997
2,32,"손담비, 출산 25일 만에 외출…붓기 없이 청바지 ‘완벽핏’ [MD★스타]",http://v.daum.net/v/20250506155104793
3,33,"하이키 옐, ‘톡식’ 커버 영상 화제…흰 티+청바지로도 ‘핫해’",http://v.daum.net/v/20250430145626938
4,34,"효민, 신행 다녀와서 ♥S대 금융맨家 찾았나? 흰티+청바지 청순룩 컴백",http://v.daum.net/v/20250430135410662


In [37]:
result1_df = pd.DataFrame(result1)
result1_df.head()

,no,title,link
0,0,사천에서 쾌청한 날씨 속 탁 트인 남해 풍경 즐겨요!,http://v.daum.net/v/20250528104429529
1,1,"윤승아, ♥김무열 쏙 빼닮은 깜찍 子 하객룩 공개‥벌써 우월한 유전자",http://v.daum.net/v/20250526145941690
2,2,"이번 주 포천 여행,반팔 하나로 충분할까?",http://v.daum.net/v/20250527171732518
3,3,"이재명, 첫 대학 방문…이준석 겨냥?",http://v.daum.net/v/20250526191942751
4,4,"""나는 젊어"" 팔소매 접은 이준석의 '흰셔츠' 속에도 뼈가 담겼다[21대 대선 리...",http://v.daum.net/v/20250525070013619


In [48]:
result0_df.to_csv('data/ch14_'+keywords[0]+'.csv', index=False, encoding='cp949')
result1_df.to_csv('data/ch14_'+keywords[1]+'.csv', index=False) # utf-8

### 4) user-agent 를 추가하여 크롤링

- urlopen() 함수를 사용하면 크롤링이 안 되는 사이트
- User-agent 를 추가하여 크롤링 ex) Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36

In [46]:
# 방법2
from urllib.request import urlopen, Request
import urllib.parse
import pandas as pd
from bs4 import BeautifulSoup

word = '비트코인'
word = urllib.parse.quote(word)

url = "https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q="+word+"&p=2"
# print(url)

# User-Agent 를 추가하여, 브라우저처험 보이게 포장
headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36'}

request = Request(url, headers=headers)
response = urlopen(request)
response.status

soup = BeautifulSoup(response.read(),'html.parser')

def collect_list(keyword, page=1):
    'keyword로 다음 검색한 결과(해당 page)를 return'
    url = f"https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y"
    params = {'q':keyword,'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text,'html.parser')

    items_find_list = [] # 검색한 결과를 담는 2차원 리스트
    item_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')

    for idx, item in enumerate(item_el):
        items_find_list.append({'no': (page-1)*10+idx, 'title':item.text, 'link':item.attrs.get('href')})

    return items_find_list


collect_list('백종원',3)

[{'no': 20,
  'title': ' 백종원 빽다방, 내달 12일까지 음료 할인…비용 본사가 부담 ',
  'link': 'http://v.daum.net/v/20250527082714508'},
 {'no': 21,
  'title': ' K푸드 수출, 이젠 소스다…불닭 넘는 다음 한 수는 ',
  'link': 'http://v.daum.net/v/20250528140952873'},
 {'no': 22,
  'title': ' 쏟아지는 악재 악재...매출 45% 급감 끝없는 ‘백종원 리스크’ ',
  'link': 'http://v.daum.net/v/20250526113905209'},
 {'no': 23,
  'title': ' "백종원 대놓고 사기 쳐" 연이은 악재에 또...가맹점 매출 \'급감\' [지금이뉴스] ',
  'link': 'http://v.daum.net/v/20250526111905124'},
 {'no': 24,
  'title': ' 백종원 빽다방, 릴레이 할인 진행…"가맹점 부담 NO" ',
  'link': 'http://v.daum.net/v/20250527081520232'},
 {'no': 25,
  'title': ' 빽다방, 할인 행사…"본사가 비용 전액 부담해 가맹점 매출 증대" ',
  'link': 'http://v.daum.net/v/20250527090303655'},
 {'no': 26,
  'title': ' 빽다방, 아메리카노 500원에 판다…할인 프로모션 내달 12일까지 ',
  'link': 'http://v.daum.net/v/20250527185546889'},
 {'no': 27,
  'title': " 백종원 리스크에 가맹점주들 '직격타'…매출 20% 급감 ",
  'link': 'http://v.daum.net/v/20250526090415260'},
 {'no': 28,
  'title': ' "아아 500원, 라떼 1000원" 빽다방 릴레이 할인 ',
  'link': 'http:

- ★ 꼭 User-Agent를 사용하여햐 하는 경우 : https://www.melon.com/chart/index.htm

In [51]:
# 방법1
import requests
from bs4 import BeautifulSoup

url = "https://www.melon.com/chart/index.htm"

melon_page = requests.get(url)
melon_page.status_code # 406 페이지가 막혀있는 코드

406

In [57]:
# 방법2
from urllib.request import urlopen
url = "https://www.melon.com/chart/index.htm"

melon_page = urlopen(url) # 에러남
melon_page

HTTPError: HTTP Error 406: Not Acceptable

In [98]:
# user-agent 추가
import requests
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import time
url = "https://www.melon.com/chart/index.htm"
headers = {"user-agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36"}

# 방법2
# request = Request(url, headers=headers)
# melon_page = urlopen(request)
# melon_page.status

# soup = BeautifulSoup(melon_page,'html.parser')

# 방법1
melon_page = requests.get(url, headers=headers)
melon_page.status_code

soup = BeautifulSoup(melon_page.text,'html.parser')
# soup


In [110]:
# 멜론 순위  노래제목,       가수  
# 1위        너에게 달기를   10cm  

title = soup.select('table div.ellipsis > span:nth-child(1) > a')

# [t.text.strip() for t in title]

writer = soup.select('table div.ellipsis > span.checkEllipsis')
# [w.text for w in write]


result = []

# for idx, t in enumerate(title):
#     print(f'{idx}')
#     result.append({'멜론순위': idx+1,"노래제목":title[idx].text, "가수":writer[idx].text})
# #     time.sleep(1)

for idx, (t, w) in enumerate(zip(title, writer)):
    result.append({
        '멜론순위':idx+1, "노래제목":t.text, "가수":w.text
    })
# result
melongData = pd.DataFrame(result)
melongData.head(18)

,멜론순위,노래제목,가수
0,1,너에게 닿기를,10CM
1,2,Never Ending Story,아이유
2,3,Drowning,WOODZ
3,4,like JENNIE,제니 (JENNIE)
4,5,모르시나요(PROD.로코베리),조째즈
5,6,THUNDER,세븐틴 (SEVENTEEN)
6,7,TOO BAD (feat. Anderson .Paak),G-DRAGON
7,8,네모의 꿈,아이유
8,9,Whiplash,aespa
9,10,"HOME SWEET HOME (feat. 태양, 대성)",G-DRAGON
